In [1]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)


download_and_unzip("https://files.grouplens.org/datasets/movielens/ml-100k.zip",
                   extract_to='.')

Next, let's import all of the modules that we'll use in this notebook.

In [2]:
# Standard library imports
import random
import time

# Third-party imports
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.notebook import tqdm
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def make_dfs():
  columns_name=['user_id','item_id','rating','timestamp']
  train_df = pd.read_csv("./ml-100k/ua.base",sep="\t",names=columns_name)
  test_df = pd.read_csv("./ml-100k/ua.test",sep="\t",names=columns_name)

  # df = df[df['rating']>=3]

  # train, test = train_test_split(df.values, test_size=0.1, random_state=16)
  # train_df = pd.DataFrame(train, columns=df.columns)
  # test_df = pd.DataFrame(test, columns=df.columns)


  return train_df, test_df

In [71]:
def preproc(train_df, test_df):
    film_columns = ["item_id", "movie title", "release date", "video release date",
              "IMDb URL", "unknown", "Action", "Adventure", "Animation",
              "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
              "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
              "Thriller", "War", "Western"]

    films_df = pd.read_csv("./ml-100k/u.item", sep="|", names=film_columns, encoding='latin-1')
    films_df.drop(["movie title", "release date", "IMDb URL", "unknown", "video release date"], axis = 1, inplace = True)

    train_df = pd.merge(train_df, films_df, how='left', left_on='item_id', right_on='item_id')
    test_df = pd.merge(test_df, films_df, how='left', left_on='item_id', right_on='item_id')

    user_columns = ["user_id", "age", "sex", "occupation", "zip_code"]
    user_df = pd.read_csv("./ml-100k/u.user", sep="|", names=user_columns, encoding='latin-1')
    user_df["sex"] = pp.LabelEncoder().fit_transform(user_df["sex"])
    occup_df = pd.read_csv("./ml-100k/u.occupation", sep="\t", names=["jobs"])
    le = pp.LabelEncoder()
    le.fit(occup_df["jobs"])
    user_df["occupation"] = le.transform(user_df["occupation"])
    user_df.drop(["zip_code"], axis = 1, inplace = True)

    train_df = pd.merge(train_df, user_df, how='left', left_on='user_id', right_on='user_id')
    test_df = pd.merge(test_df, user_df, how='left', left_on='user_id', right_on='user_id')
    
    
    train_df["rating"]  = train_df["rating"] - 1
    train_df["age"] = train_df["age"] / user_df["age"].abs().max()
    
    encoder = pp.OneHotEncoder(sparse_output=False)
    encoder.fit(train_df[["occupation"]])
    new_feat = encoder.transform(train_df[["occupation"]])
    new_cols = pd.DataFrame(new_feat, columns=encoder.get_feature_names_out(["occupation"]))
    train_df = pd.concat([train_df, new_cols], axis=1)    
    train_df.drop("occupation", axis=1, inplace=True)
    
    new_feat = encoder.transform(test_df[["occupation"]])
    new_cols = pd.DataFrame(new_feat, columns=encoder.get_feature_names_out(["occupation"]))
    test_df = pd.concat([test_df, new_cols], axis=1)    
    test_df.drop("occupation", axis=1, inplace=True)
    
    test_df["rating"]  = test_df["rating"] - 1
    test_df["age"] = test_df["age"] / user_df["age"].abs().max()

    train_df.drop(["item_id", "user_id", "timestamp"], axis = 1, inplace = True)
    train_y = train_df["rating"].values
    train_x = train_df.drop('rating', axis=1).values

    test_df.drop(["item_id", "user_id", "timestamp"], axis = 1, inplace = True)
    test_y = test_df["rating"].values
    test_x = test_df.drop('rating', axis=1).values
    
#     return train_df, test_df

    return train_x, train_y, test_x, test_y


In [73]:
train_df, test_df = make_dfs()
train_x, train_y, test_x, test_y = preproc(train_df, test_df)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [74]:
class LensDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
      self.x = x
      self.y = y

    def __getitem__(self, index):
      features = self.x[index]
      rating = self.y[index]

      features = torch.tensor(features, dtype=torch.float32)
      rating = torch.tensor(rating, dtype=torch.int64)
      return (features, rating)

    def __len__(self):
        return len(self.y)

In [75]:
train_dataset = LensDataset(train_x, train_y)
test_dataset = LensDataset(test_x, test_y)

In [76]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=batch_size, shuffle=False)
loaders = {
    'train' : train_loader,

    'test'  : test_loader,
}

batch = next(iter(train_loader))

print(batch[0].shape, batch[1].shape) # Example of data shapes

tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.2603, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000]) torch.Size([128])


In [77]:
class Net(nn.Module):
    def __init__(self, inp_size, out_size):
        super(Net, self).__init__()

        self.linear_layers = nn. Sequential(
            nn.Linear(inp_size, 1024, bias=True),
            nn.ReLU(),
            nn.Linear(1024, 4096, bias=True),
            nn.ReLU(),
            nn.Linear(4096, 8192, bias=True),
            nn.ReLU(),
            nn.Linear(8192, 4096, bias=True),
            nn.ReLU(),
            nn.Linear(4096, 256, bias=True),
            nn.ReLU(),
            nn.Linear(256, out_size, bias=True),
            nn.Softmax(),
        )


    def forward(self, x):
        out = self.linear_layers(x)
        return out

In [81]:
from torch.optim import lr_scheduler
import copy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

inp_size = 41
out_size=5
num_epochs = 40

model = Net(inp_size, out_size).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.00005)

best_model_wts = copy.deepcopy(model.state_dict()) # Best weights
best_loss = 10**10

model

Net(
  (linear_layers): Sequential(
    (0): Linear(in_features=41, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=8192, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8192, out_features=4096, bias=True)
    (7): ReLU()
    (8): Linear(in_features=4096, out_features=256, bias=True)
    (9): ReLU()
    (10): Linear(in_features=256, out_features=5, bias=True)
    (11): Softmax(dim=None)
  )
)

In [82]:
from time import time
from tqdm import tqdm
# Train and test functions
def train(model, device, criterion, optimizer, epoch):
    model.train()
    epoch_loss = 0
    start_time = time()
    correct = 0
    iteration = 0

    bar = tqdm(loaders['train'])
    for data, target in bar:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
#         print(output)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        iteration += 1
        bar.set_postfix({"Loss": format(epoch_loss/iteration, '.6f')})

    print(f'Train Epoch: {epoch}, elapsed time:{time()-start_time:.2f}s, Avg train Loss: {epoch_loss/iteration}')
    return epoch_loss / iteration


def test(model, device, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    iters = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            iters += 1
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = torch.argmax(output, dim=1)
            if correct == 0:
                print(pred, target)
            correct += sum(pred.eq(target))
            total += len(pred)
            test_loss += criterion(output, target).item()  # sum up batch loss

    test_loss /= iters
    print(f'Avg test Loss: {test_loss}, Avg Accuracy: {correct/total}')
    return test_loss

In [83]:
for _ in range(num_epochs):
    train_loss = train(model, device, criterion, optimizer, _)
    test_loss = test(model, device, criterion)

    if test_loss < best_loss: # Keep the best weights
        best_loss = test_loss
        best_model_wts = copy.deepcopy(model.state_dict())

  0%|          | 0/708 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 708/708 [00:28<00:00, 24.56it/s, Loss=1.525078]


Train Epoch: 0, elapsed time:28.83s, Avg train Loss: 1.5250783025544916
tensor([3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 4, 2, 3, 3, 2, 3, 3, 3, 2, 3, 2, 2, 3,
        2, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 2,
        2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 4,
        3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 4, 3, 2, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')
Avg test Loss: 1.5180680977331626, Avg Accuracy: 0.3492046594619751


100%|██████████| 708/708 [00:28<00:00, 24.57it/s, Loss=1.515797]


Train Epoch: 1, elapsed time:28.81s, Avg train Loss: 1.5157972774599906
tensor([3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 4, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 2, 3,
        2, 3, 2, 2, 3, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 2, 2,
        2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 4,
        2, 3, 3, 2, 2, 4, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 3, 3, 3, 4, 2, 2, 4, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')
Avg test Loss: 1.5137723426561098, Avg Accuracy: 0.35079532861709595


100%|██████████| 708/708 [00:28<00:00, 24.61it/s, Loss=1.511222]


...

Train Epoch: 37, elapsed time:28.78s, Avg train Loss: 1.4586843184832126
tensor([3, 3, 3, 2, 2, 3, 3, 4, 2, 3, 2, 3, 2, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 2,
        3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 4, 2, 2, 2, 2,
        2, 3, 2, 2, 4, 3, 2, 4, 3, 4, 3, 3, 3, 3, 4, 4, 2, 3, 4, 3, 3, 2, 3, 3,
        3, 3, 3, 2, 2, 3, 2, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 3, 3,
        3, 4, 4, 3, 2, 2, 2, 2, 2, 4, 2, 2, 2, 3, 2, 2, 3, 4, 3, 2, 3, 2, 4, 3,
        3, 3, 2, 3, 3, 3, 3, 3], device='cuda:0')
Avg test Loss: 1.5157281031479706, Avg Accuracy: 0.3633085787296295


100%|██████████| 708/708 [00:28<00:00, 24.55it/s, Loss=1.457768]


Train Epoch: 38, elapsed time:28.84s, Avg train Loss: 1.457768476615518
tensor([3, 3, 3, 2, 2, 3, 3, 4, 2, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2,
        3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 4, 2, 2, 3, 2,
        2, 3, 3, 2, 4, 3, 2, 4, 3, 4, 3, 3, 3, 2, 4, 4, 2, 3, 3, 2, 3, 2, 3, 3,
        3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 3, 3,
        3, 3, 4, 3, 2, 2, 2, 2, 2, 4, 3, 2, 2, 3, 2, 3, 3, 4, 4, 2, 3, 2, 4, 3,
        3, 3, 2, 3, 3, 3, 3, 3], device='cuda:0')
Avg test Loss: 1.5165064850369014, Avg Accuracy: 0.363838791847229


100%|██████████| 708/708 [00:28<00:00, 24.59it/s, Loss=1.456863]


Train Epoch: 39, elapsed time:28.79s, Avg train Loss: 1.4568627666955614
tensor([3, 3, 3, 2, 2, 3, 3, 4, 2, 2, 2, 3, 2, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 2,
        2, 3, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 4, 2, 2, 2, 2,
        2, 3, 3, 2, 4, 3, 2, 4, 3, 4, 3, 3, 3, 2, 4, 4, 2, 3, 4, 2, 3, 2, 3, 3,
        2, 4, 2, 2, 2, 3, 2, 2, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 3, 3,
        3, 4, 4, 3, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 3, 4, 3, 2, 2, 2, 4, 3,
        3, 3, 2, 3, 3, 3, 3, 3], device='cuda:0')
Avg test Loss: 1.5184849355671857, Avg Accuracy: 0.3556733727455139
